In [37]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np
from JoyStick import Joystick
from Background import Background
#from Character import Character


In [38]:
class Character:
  def __init__(self,life):
    # 캐릭터에 사용할 이미지
    self.cat_front = Image.open('images/character/cat_front.png').convert('RGBA')
    self.cat_left = Image.open('images/character/cat_left.png').convert('RGBA')
    self.cat_right = Image.open('images/character/cat_right.png').convert('RGBA')
    self.cat_angry = Image.open('images/character/cat_angry.png').convert('RGBA')
    # 이미지 크기 줄이기 
    self.cat_front_resized = self.cat_front.resize((70, 70)) 
    self.cat_left_resized = self.cat_left.resize((70, 70)) 
    self.cat_right_resized = self.cat_right.resize((70, 70)) 
    self.cat_angry_resized = self.cat_angry.resize((70, 70)) 
    # 캐릭터 위치
    self.x_position = 120
    self.y_position = 180
    self.width = 50
    self.height = 50

    self.speed = 5
    self.state= None

    # 캐릭터 목숨
    self.life = life
    self.life_list=[]
    # 목숨은 3개, 리스트에 이미지를 넣는다.
    self.life_image = Image.open('./images/character/heart.png').convert('RGBA')
    self.life_image_resized = self.life_image.resize((20,20))
    self.life_list.append(self.life_image_resized)
    self.life_list.append(self.life_image_resized)
    self.life_list.append(self.life_image_resized)

    self.score = 0

   # 위 아래로는 움직이지 않음 
  def move(self,up_down,left_right):
      if up_down=='up_pressed':
           pass
      elif up_down == 'down_pressed':
           pass
      elif left_right == 'left_pressed':
          self.x_position -= self.speed
      elif left_right == 'right_pressed':
          self.x_position += self.speed

In [39]:
class Fish:
  def __init__(self,x1,y1,x2,y2,stage,speed):
    self.fish_1 = Image.open('images/character/fish.png').convert('RGBA')
    self.fish2 = Image.open('images/character/fish2.png').convert('RGBA')
    self.fish_resized = self.fish_1.resize((50, 50)) 
    self.fish2_resized = self.fish2.resize((40, 40))
    #물고기가 내려오는 속도
    self.speed = speed 
    # 물고기의 위치
    self.x1=x1
    self.y1=y1
    self.x2=x2
    self.y2=y2
    self.col_count = 0
    self.is_collided = False
    self.stage=stage
  
  def descend(self):
      if self.y1 < 240 :
          self.y1 += self.speed
          self.y2 += self.speed
      else:
            self.y1 = 0
            self.y2 = 50  # Assuming fish height is 50
            self.x1 = random.randrange(0, 240)
            self.x2 = self.x1 + 20

  def collision_check(self, cat):
    if (
        (cat.x_position + 40 >= self.x1 and 
        cat.x_position  <  self.x2 and 
        cat.y_position  >= self.y1 and 
        cat.y_position + 35 < self.y2) 
    ):
        return 1  
    else:
        return 0 
                
  def speed_up(self):
        self.speed += 100
        
  def reset_collision(self):
        self.is_collided = False            

In [40]:
class Fishes:
  def __init__(self, stage):
    self.fish_list = []
    self.stage = stage
    self.fishTime = time.time()  # 최초 생성 시간 기록
    self.max_fish = 10  # 최대 뼈다귀 개수
    self.speed = 5
    

  def spawn_fish(self):
        if len(self.fish_list) < self.max_fish:
            x1 = random.randrange(0, 240)
            y1 = 0
            x2 = x1 + 20
            y2 = y1 + 50
            self.fish_list.append(Fish(x1, y1, x2, y2,self.stage,self.speed))

  def update(self):
        current_time = time.time() # 현재시간 업데이트
        timeGap = current_time - self.fishTime# 물고기를 처음 생성한 시간 - 지금 시간

        if timeGap > 1:  # 1초마다 뼈다귀 생성
            self.spawn_fish()
            self.timeGap = current_time # 다시 초기화

        for fish in self.fish_list:
            fish.descend() # 

            if fish.y1 >= 240:
                self.fish_list.remove(fish)  # 뼈다귀가 화면을 벗어나면 리스트에서 제거

  def draw(self, image):
        for fish in self.fish_list: 
            x_position = fish.x1
            y_position = fish.y1
            image.paste(fish.image_resized, (x_position, y_position), fish.image_resized)
  


In [41]:
class Bone:
  def __init__(self,x1,y1,x2,y2,speed):
     self.image = Image.open('images/character/bone.png').convert("RGBA")
     self.image_resized = self.image.resize((50, 50))
     self.x1=x1 
     self.y1=y1 
     self.x2=x2 
     self.y2=y2 
     self.is_collided = False
     self.speed = speed

  def descend(self):
        #y만 self.speed 만큼 + 하면서 내려오게 하면 됨
      if self.y1 < 240 :
          self.y1 = self.y1 + self.speed
          self.y2 = self.y2 + self.speed
      else:
            self.y1 = 0
            self.y2 = 50
            self.x1 = random.randrange(0, 240)
            self.x2 = self.x1 + 10
  
  def set_speed(self, new_speed):
        self.speed = new_speed

  def collision_check(self, cat):
    if (
        (cat.x_position + 40 >= self.x1 and 
        cat.x_position  <  self.x2 and 
        cat.y_position  >= self.y1 and 
        cat.y_position + 35 < self.y2) 
    ):
        return 1  # Collision detected
    else:
        return 0  # No collision
        
  



In [42]:
class Bones:
  def __init__(self,stage,screen_width):
    self.bone_list = []
    self.stage = stage
    self.screen_width = screen_width
    self.last_bone_spawn_time = time.time()  # 최초 생성 시간 기록
    self.max_bones = 10  # 최대 뼈다귀 개수
    self.bone_speed = 5

    self.stage_speeds = {1: 5, 2: 7, 3: 9, 4: 11, 5: 13}

    self.spawn_bone()  # 초기에 뼈다귀 생성
  
  def spawn_bone(self):
        if len(self.bone_list) < self.max_bones:
            x1 = random.randrange(0, self.screen_width - 50)
            y1 = 0
            x2 = x1 + 20
            y2 = y1 + 50
            self.bone_list.append(Bone(x1, y1, x2, y2,self.bone_speed))

  def update(self):
        current_time = time.time()
        time_elapsed_since_last_spawn = current_time - self.last_bone_spawn_time

        if time_elapsed_since_last_spawn > 1:  # 1초마다 뼈다귀 생성
            self.spawn_bone()
            self.last_bone_spawn_time = current_time

        for bone in self.bone_list:
            bone.descend()

            if bone.y1 >= 240:
                self.bone_list.remove(bone)  # 뼈다귀가 화면을 벗어나면 리스트에서 제거

  def draw(self, image):
        for bone in self.bone_list: 
            x_position = bone.x1
            y_position = bone.y1
            image.paste(bone.image_resized, (x_position, y_position), bone.image_resized)

In [43]:
life = 3
hit=0
left_right = 'None'

k=0
score= 0
new_stage = 1
fish_index = [0, 1, 2, 3, 4, 5, 6, 7]

# joystick 객체
joystick = Joystick()
background = Background()

# 캐릭터 객체 생성
cat = Character(life)
delete_cat = [] #지워지는 적들 index 리스트에 저장

stage = 1

# 물고기 객체 생성
fishes = Fishes(stage)
bones = Bones(stage, joystick.width)


# Get drawing object to draw on image
image = Image.new("RGBA", (joystick.width, joystick.height))
draw = ImageDraw.Draw(image)
# clear display
draw.rectangle((0, 0, joystick.width, joystick.height), outline=0, fill=(255, 0, 0))
joystick.disp.image(image)


# 폰트 설정하기
fnt1 = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 20)
fnt2 = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 50)


In [44]:
#background = Image.open('images/background/background1.png')
#background = background.resize((240,240))
# for i in range(10):
#   background = Background().backgroundList[i]
#   background = background.resize((240,240))
total_backgrounds=len(background.backgroundList)
current_background_index = 0



In [45]:

while True:
    up_down = 'None'
    left_right = 'None'
    
    if not joystick.button_U.value:
        up_down = 'up_pressed'
    if not joystick.button_D.value:
        up_down = 'down_pressed'
    if not joystick.button_L.value:
        left_right = 'left_pressed'
    if not joystick.button_R.value:
        left_right = 'right_pressed'
    if not joystick.button_A.value:
        game_start = 'game_start'

    time.sleep(0.01)
    current_background_index = (current_background_index + 1) % 10
    current_background = background.backgroundList[current_background_index]
    current_background = current_background.resize((240, 240))
    image.paste(current_background, (0, 0),current_background)


    # 목숨 몇개 남았는 지 보여주기, 만약 hit이 생겼다면 리스트에서 하나씩 빼면 됨 -> 나중에 구현해보자
    # 3개의 목숨이 다 보이게 할려면 이미지가 붙을 위치를 다르게 해야하기 때문에 j의 값을 바꿔야함
    j=0
    for i in range(cat.life):
        image.paste(cat.life_list[i],(j,0),cat.life_list[i])
        j+=15

    if(cat.life<=0):
        while True:
         rcolor = tuple(int (x * 255) for x in hsv_to_rgb(random.random(), 1, 1))
         draw.text((40, 70), "Game", font = fnt2, fill = rcolor)
         draw.text((40, 110), "Over", font = fnt2, fill = rcolor)
         joystick.disp.image(image)
    


    # for i in range(len(fishes.fish_list)):
    #     if i < len(fishes.fish_list):  # Check if i is a valid index
    #         fishes.fish_list[i].descend()
    #         if fishes.fish_list[i].y1 == 0:
    #             new_fish = Fish(random.randrange(0, 240), 0, random.randrange(0, 240), 0, fishes.fish_list[i].stage, fishes.fish_list[i].speed)
    #             fishes.fish_list[i] = new_fish
    #         x_position = fishes.fish_list[i].x1
    #         y_position = fishes.fish_list[i].y1
    #         image.paste(fishes.fish_list[i].fish_resized, (x_position, y_position), fishes.fish_list[i].fish_resized)

    fishes.update()
    fishes.draw(image)
    bones.update()
    bones.draw(image)
         
        
    # 움직이게 만들기 
    cat.move(up_down, left_right)
    # 가만히 있을 때 이미지표시
    image.paste(cat.cat_front_resized, (cat.x_position,cat.y_position),cat.cat_front_resized)
    # 왼쪽 오른쪽으로 움직일 때 다른 이미지로 보이도록 함.
    if(left_right =='left_pressed') :
        image.paste(cat.cat_left_resized, (cat.x_position,cat.y_position),cat.cat_left_resized)
    elif(left_right == 'right_pressed'):
         image.paste(cat.cat_right_resized, (cat.x_position,cat.y_position),cat.cat_right_resized)
    
    # for fish in fishes:
    #     image.paste(fish,(0,0),fish)
    draw.text((0, 10), "score:", font = fnt1, fill = (255,255,255))
    draw.text((60, 10), str(score), font = fnt1, fill = (255,255,255))

    # 물고기와의 충돌 체크
    # 충돌 되면 score 점수가 올라감
    # score점수가 100이 되면 stage가 하나 올라감
    for fish in fishes.fish_list:
        if not fishes.fish_list[i].is_collided:  # 이미 충돌이 확인된 물고기는 무시
            get_score = fish.collision_check(cat)
            if get_score == 1:
                score +=10
                fishes.fish_list[i].is_collided = True
                get_score = 0

    for bone in bones.bone_list:
        hit = bone.collision_check(cat)
        if (hit==1):    
            cat.life_list.pop()
            cat.life -= 1
            image.paste(cat.cat_angry_resized, (cat.x_position,cat.y_position),cat.cat_angry_resized)
            print("뼈다귀와 고양이 충돌!")
            bones.bone_list.remove(bone)
            hit = 0

    if(str(score) == "40"):
          stage+=1
          print("stage:",stage)
          for fish in fishes.fish_list:
              fish.speed_up()  # 물고기 속도 증가
              score = 0  # score 초기화
              if(stage==2):
                  #new_fish2 = fishes.fish_list[i]
                  pass
             
    if score == 50:
        for bone in bones.bone_list:
            bone.speed = 10  # Set the speed to 10
            score = 0



    joystick.disp.image(image)


AttributeError: 'Fish' object has no attribute 'image_resized'